In [55]:
import requests

def make_request(query):
    base_url = "http://api.conceptnet.io"
    if not query.startswith(base_url):
        query = f"{base_url}{query}"
    return requests.get(query).json()

def read_edges(obj):
    for edge in obj['edges']:
        if edge['start']['language'] == 'en' and edge['end']['language'] == 'en':
            yield {
                'edge': (edge['start']['label'], edge['rel']['label'], edge['end']['label']),
                'example': edge['surfaceText']
            }

def get_relations_between_words(word1, word2):
    word1 = word1.replace(' ', '_')
    word2 = word2.replace(' ', '_')
    query = f"/query?node=/c/en/{word1}&other=/c/en/{word2}&language=en"
    obj = make_request(query=query)
    edges = list(read_edges(obj))
    return edges

def get_edges_related_to_word(word):
    word = word.replace(' ', '_')
    query = f"/query?node=/c/en/{word}&language=en"
    obj = make_request(query = query)
    edges = list(read_edges(obj))
    return edges


In [56]:
import pandas as pd
import re
from tqdm import tqdm

dump = pd.read_csv("data/kgtk_conceptnet.tsv", sep = "\t")

dump.drop(columns=['relation;dimension', 'source'], inplace=True, axis = 1)

pattern = r"/c/en/([^/]*)(/.*)?"

from collections import defaultdict

label_dict = dict()
rel_dict = defaultdict(set)

for _, row in tqdm(dump.iterrows(), leave = False):
    node1 = row['node1']
    node2 = row['node2']

    rel = row['relation;label']
    sent = row['sentence']

    node1_id = re.match(pattern, node1).group(1)
    node2_id = re.match(pattern, node2).group(1)

    node1_label = row['node1;label']
    node2_label = row['node2;label']
    
    label_dict[node1_id] = node1_label
    label_dict[node2_id] = node2_label

    rel_dict[(node1_id, node2_id)].add((rel, sent))

    

/tmp/ipykernel_3913714/3493011625.py:5: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  dump = pd.read_csv("data/kgtk_conceptnet.tsv", sep = "\t")


In [ ]:
node1 = 'bank_of_china'
node2 = 'bank'

for rel in rel_dict[(node1, node2)]:
    print(f"{label_dict[node1]} {rel[0]} {label_dict[node2]}")

In [68]:
import joblib

joblib.dump(rel_dict, "data/conceptNet_relations.joblib")
joblib.dump(label_dict, "data/conceptNet_label.joblib")

['data/conceptNet_label.joblib']

In [3]:
get_relations_between_words('stepfather', 'father')

[{'edge': ('stepfather', 'DerivedFrom', 'father'), 'example': None},
 {'edge': ('stepfather', 'IsA', 'father'), 'example': None},
 {'edge': ('stepfather', 'DerivedFrom', 'father'), 'example': None}]

In [4]:
get_relations_between_words('bank', 'chase')

[{'edge': ('chase', 'IsA', 'bank'), 'example': None}]

In [5]:
get_relations_between_words('bank', 'bank of china')

[{'edge': ('bank of china', 'IsA', 'bank'), 'example': None}]

In [6]:
get_relations_between_words('bank', 'teller')

[{'edge': ('teller', 'RelatedTo', 'bank'), 'example': None},
 {'edge': ('bank', 'RelatedTo', 'teller'),
  'example': '[[bank]] is related to [[teller]]'}]